# Clip Tokenizer



In [1]:
#@title 1. Install Requirements

print("This will take a couple of minutes...")
!pip install -q --upgrade diffusers[torch]
#!pip install -q xformers==0.0.17
!pip install -q transformers
#!pip install -q triton



This will take a couple of minutes...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.4 MB/s eta 0:00:00


In [2]:
#@title 2. Setup Function

from transformers import CLIPTokenizer, CLIPModel
import json


# ===========================================================================
# ===========================================================================
def tokenize(token):

  with open(Vocab_File_Path, "r", encoding='utf-8') as f:
    vocab = json.load(f)
    vocab = {v: k for k, v in vocab.items()}

  tokenizer = CLIPTokenizer.from_pretrained(MODEL_PATH, subfolder="tokenizer",revision="fp16")

  # print(tokenizer)
  print(f'vocab size: {tokenizer.vocab_size}')
  print(f'max length: {tokenizer.model_max_length}')
  print(f'special token (bos): {tokenizer.bos_token}')
  print(f'special token (eos): {tokenizer.eos_token}')
  print(f'special token (unk): {tokenizer.unk_token}')
  print(f'special token (pad): {tokenizer.pad_token}\n')

  #inputs = tokenizer(sys.argv[1], padding=True)
  inputs = tokenizer(f'{token}', padding=True)
  value = sum(inputs["input_ids"])
  value_str = str(value)
  token_data = ''

  # holds lines of sub-tokens up to the width of the grid (minus margin)
  return_list = []
  margin_right = 256

  # add current string to list
  #return_list.append(token_data)
  #token_data = '' # reset

  # print(inputs["input_ids"])
  #  print the token ids in rows of 10
  row_length = 10
  for i in range(0, len(inputs["input_ids"]), row_length):
      # Extract a row's worth of numbers
      row = inputs["input_ids"][i:i+row_length]
      # Create a formatted string for the row
      row_string = ', '.join(f"{num:5d}" for num in row)
      # Print the row without the trailing comma on the last row and without a newline at the end
      print(row_string + (',' if i + row_length < len(inputs["input_ids"]) else ''), end='\n')



  print('\n')

  token_count = 0
  for x in range(len(inputs["input_ids"])):
    #print(vocab[inputs["input_ids"][x]] + " " + str(inputs["input_ids"][x]))



    # don't include <|startoftext|> 49406 or <|endoftext|> 49407
    # if inputs["input_ids"][x] != 49406 and inputs["input_ids"][x] != 49407:
    # add next sub-token
    token_data += '[' + vocab[inputs["input_ids"][x]] + ']'
    token_count += 1
    # check string length, if >= grid_width
    # if (get_text_size(token_data, myFont)[0] + margin_right) >= grid_width:
    #   return_list.append(token_data.strip())
    #   token_data = '' #reset string
    if x < len(inputs["input_ids"]) - 2:
      token_data += ' '

  # add last or only string
  if len(token_data.strip()) > 0:
    return_list.append(token_data.strip())

  # print(return_list)
  print("Token Count = " + f'{token_count}')


  return return_list, token_count



In [7]:
#@title 3. Select Model
# =========================================================================================
# SETUP SD PIPELINE
# =========================================================================================
import torch
import os
import shutil
from diffusers import StableDiffusionPipeline, AutoencoderKL, DDIMScheduler

if torch.cuda.is_available():
    torch.cuda.empty_cache()


SELECT_MODEL = 'Stable Diffusion 1.5' #@param ["Stable Diffusion 1.5", "Stable Diffusion 2.1"]

# can be local paths also
if SELECT_MODEL=='Stable Diffusion 1.5':
  MODEL_PATH = "runwayml/stable-diffusion-v1-5"
  VAE_PATH = "stabilityai/sd-vae-ft-mse"
  pipe = StableDiffusionPipeline.from_pretrained(f'{MODEL_PATH}',
                                               vae = AutoencoderKL.from_pretrained(f'{VAE_PATH}', torch_dtype = torch.float16),
                                               revision="fp16",
                                               torch_dtype = torch.float16,
                                               safety_checker = None)
elif SELECT_MODEL=='Stable Diffusion 2.1':
  MODEL_PATH = "stabilityai/stable-diffusion-2-1"
  pipe = StableDiffusionPipeline.from_pretrained(f'{MODEL_PATH}',
                                               revision="fp16",
                                               torch_dtype = torch.float16,
                                               safety_checker = None)



pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

# =========================================================================================
# Copy the vocab.json from the cache to current dir
# =========================================================================================
# tokenizer/vocab.json files are normally saved in cache e.g.
# /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/xxxxx/tokenizer/vocab.json

name = 'vocab.json'
init_path = '/root/.cache/'
Vocab_File_Path = None  # Initialize the variable to store the vocab file path

# Walk through the directory structure starting at init_path
for root, dirs, files in os.walk(init_path):
    if name in files:
        # If the vocab.json file is found, store its path
        Vocab_File_Path = os.path.join(root, name)
        print(f"Found vocab file at: {Vocab_File_Path}")
        break  # Exit the loop once the file is found

if Vocab_File_Path is not None:
    print("Vocab file path set.")
else:
    print("Vocab file not found.")

# Now Vocab_File_Path contains the path to the vocab.json file

unet/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Found vocab file at: /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/ded79e214aa69e42c24d3f5ac14b76d568679cc2/tokenizer/vocab.json
Vocab file path set.


In [8]:
TOKENS = "We’re introducing a neural network called CLIP which efficiently learns visual concepts from natural language supervision. CLIP can be applied to any visual classification benchmark by simply providing the names of the visual categories to be recognized, similar to the “zero-shot” capabilities of GPT-2 and GPT-3. called CLIP which efficiently learns visual concepts from natural language supervision. CLIP can be applied to any called CLIP which efficiently learns visual concepts from natural language supervision. CLIP can be applied to any"

tokenize(TOKENS)

Token indices sequence length is longer than the specified maximum sequence length for this model (102 > 77). Running this sequence through the model will result in indexing errors


vocab size: 49408
max length: 77
special token (bos): <|startoftext|>
special token (eos): <|endoftext|>
special token (unk): <|endoftext|>
special token (pad): <|endoftext|>

49406,   649,   728,   503,   810,  6256,   320, 26388,  3304,  2726,
 9289,  1448, 32915, 17569,  7195, 16763,   633,  3288,  4781, 36234,
  269,  9289,   753,   655, 12636,   531,  1504,  7195, 26612, 31775,
  638,  6151,  7250,   518,  6130,   539,   518,  7195, 19971,   531,
  655,  9929,   267,  8547,   531,   518,   728,   506,  5848,   268,
 2000,   728,   507, 19512,   539,    70,  1459,   268,   273,   537,
   70,  1459,   268,   274,   269,  2726,  9289,  1448, 32915, 17569,
 7195, 16763,   633,  3288,  4781, 36234,   269,  9289,   753,   655,
12636,   531,  1504,  2726,  9289,  1448, 32915, 17569,  7195, 16763,
  633,  3288,  4781, 36234,   269,  9289,   753,   655, 12636,   531,
 1504, 49407


Token Count = 102


(['[<|startoftext|>] [we</w>] [âĢ] [Ļ</w>] [re</w>] [introducing</w>] [a</w>] [neural</w>] [network</w>] [called</w>] [clip</w>] [which</w>] [efficiently</w>] [learns</w>] [visual</w>] [concepts</w>] [from</w>] [natural</w>] [language</w>] [supervision</w>] [.</w>] [clip</w>] [can</w>] [be</w>] [applied</w>] [to</w>] [any</w>] [visual</w>] [classification</w>] [benchmark</w>] [by</w>] [simply</w>] [providing</w>] [the</w>] [names</w>] [of</w>] [the</w>] [visual</w>] [categories</w>] [to</w>] [be</w>] [recognized</w>] [,</w>] [similar</w>] [to</w>] [the</w>] [âĢ] [ľ</w>] [zero</w>] [-</w>] [shot</w>] [âĢ] [Ŀ</w>] [capabilities</w>] [of</w>] [g] [pt</w>] [-</w>] [2</w>] [and</w>] [g] [pt</w>] [-</w>] [3</w>] [.</w>] [called</w>] [clip</w>] [which</w>] [efficiently</w>] [learns</w>] [visual</w>] [concepts</w>] [from</w>] [natural</w>] [language</w>] [supervision</w>] [.</w>] [clip</w>] [can</w>] [be</w>] [applied</w>] [to</w>] [any</w>] [called</w>] [clip</w>] [which</w>] [efficiently</w>